In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv(r'C:\Users\prati\Downloads\card_transaction.v1.csv')


In [5]:
df.shape


(24386900, 15)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24386900 entries, 0 to 24386899
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   User            int64  
 1   Card            int64  
 2   Year            int64  
 3   Month           int64  
 4   Day             int64  
 5   Time            object 
 6   Amount          object 
 7   Use Chip        object 
 8   Merchant Name   int64  
 9   Merchant City   object 
 10  Merchant State  object 
 11  Zip             float64
 12  MCC             int64  
 13  Errors?         object 
 14  Is Fraud?       object 
dtypes: float64(1), int64(7), object(7)
memory usage: 2.7+ GB


In [9]:
df.head(10)

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No
5,0,0,2002,9,3,13:53,$86.19,Swipe Transaction,-7146670748125200898,Monterey Park,CA,91755.0,5970,NaN,No
6,0,0,2002,9,4,05:51,$93.84,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
7,0,0,2002,9,4,06:09,$123.50,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
8,0,0,2002,9,5,06:14,$61.72,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
9,0,0,2002,9,5,09:35,$57.10,Swipe Transaction,4055257078481058705,La Verne,CA,91750.0,7538,NaN,No


In [11]:
df_sample= df.sample(frac=0.1, random_state=42)

In [12]:
df_sample.shape

(2438690, 15)

In [15]:
df_sample['User'].tail(20)

5592578      480
23409144    1912
20093090    1633
14619455    1198
9081229      785
14899728    1223
6967575      584
19049242    1537
13955385    1139
15997803    1299
17718584    1430
3542448      297
19010908    1535
7073331      598
2419376      201
19245832    1560
21777050    1777
22404610    1832
7428904      624
23041253    1888
Name: User, dtype: int64

In [17]:
duplicates = df_sample[df_sample.duplicated()]

print(f"Number of duplicate rows: {duplicates.shape[0]}")


Number of duplicate rows: 0


In [18]:
df_sample.nunique()

User               1999
Card                  9
Year                 30
Month                12
Day                  31
Time               1440
Amount            45470
Use Chip              3
Merchant Name     47596
Merchant City     11062
Merchant State      203
Zip               22300
MCC                 109
Errors?              20
Is Fraud?             2
dtype: int64

In [21]:
df_sample['datetime'] = pd.to_datetime(df_sample[['Year', 'Month', 'Day']].astype(str).agg('-'.join, axis=1) + ' ' + df_sample['Time'])

# 2. Extract features from the datetime column
df_sample['day_of_week'] = df_sample['datetime'].dt.dayofweek  # 0=Monday, 6=Sunday
df_sample['hour'] = df_sample['datetime'].dt.hour  # Extract hour
df_sample['is_weekend'] = df_sample['day_of_week'].isin([5, 6]).astype(int)  # 1 if weekend, else 0
df_sample['time_of_day'] = pd.cut(df_sample['hour'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'])

In [23]:
df_sample.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,datetime,day_of_week,hour,is_weekend,time_of_day
18199893,1470,0,2019,7,10,00:11,$59.18,Chip Transaction,-6853385250336487907,Harwood,MD,20776.0,5813,NaN,No,2019-07-10 00:11:00,2,0,0,NaN
9731325,822,1,2019,1,14,22:12,$280.91,Online Transaction,4241336128694185533,ONLINE,NaN,NaN,4814,NaN,No,2019-01-14 22:12:00,0,22,0,Evening
536687,41,3,2010,3,15,07:07,$-144.00,Swipe Transaction,190253443608377572,Hemet,CA,92543.0,3359,NaN,No,2010-03-15 07:07:00,0,7,0,Morning
13223840,1084,0,2015,9,20,14:58,$6.76,Chip Transaction,-7837310524365334241,Littleton,CO,80122.0,5300,NaN,No,2015-09-20 14:58:00,6,14,1,Afternoon
17070521,1384,0,2014,10,12,11:44,$9.17,Swipe Transaction,-5023497618971072366,Gardner,KS,66030.0,5812,NaN,No,2014-10-12 11:44:00,6,11,1,Morning


In [25]:
df_sample= df_sample.drop(columns=[	'Month', 'Day',	'Time', 'datetime', 'Year'])

In [27]:
df_sample.head()

,User,Card,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,day_of_week,hour,is_weekend,time_of_day
18199893,1470,0,$59.18,Chip Transaction,-6853385250336487907,Harwood,MD,20776.0,5813,NaN,No,2,0,0,NaN
9731325,822,1,$280.91,Online Transaction,4241336128694185533,ONLINE,NaN,NaN,4814,NaN,No,0,22,0,Evening
536687,41,3,$-144.00,Swipe Transaction,190253443608377572,Hemet,CA,92543.0,3359,NaN,No,0,7,0,Morning
13223840,1084,0,$6.76,Chip Transaction,-7837310524365334241,Littleton,CO,80122.0,5300,NaN,No,6,14,1,Afternoon
17070521,1384,0,$9.17,Swipe Transaction,-5023497618971072366,Gardner,KS,66030.0,5812,NaN,No,6,11,1,Morning


In [29]:
df_sample["Amount"] = df_sample["Amount"].replace({'\$': ''}, regex=True).astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
C:\Users\prati\AppData\Local\Temp\ipykernel_11416\3237195401.py:1: SyntaxWarning: invalid escape sequence '\$'
  df_sample["Amount"] = df_sample["Amount"].replace({'\$': ''}, regex=True).astype(float)


In [31]:
df_sample['Errors?'].value_counts()

Errors?
Insufficient Balance,                     24642
Bad PIN,                                   5861
Technical Glitch,                          4705
Bad Card Number,                           1356
Bad CVV,                                   1044
Bad Expiration,                            1029
Bad Zipcode,                                192
Bad PIN,Insufficient Balance,                73
Insufficient Balance,Technical Glitch,       45
Bad PIN,Technical Glitch,                    13
Bad Card Number,Insufficient Balance,        13
Bad CVV,Insufficient Balance,                 9
Bad Expiration,Bad CVV,                       8
Bad Expiration,Insufficient Balance,          7
Bad Card Number,Bad Expiration,               6
Bad Card Number,Bad CVV,                      5
Bad CVV,Technical Glitch,                     3
Bad Zipcode,Insufficient Balance,             2
Bad Expiration,Technical Glitch,              2
Bad Card Number,Technical Glitch,             2
Name: count, dtype: int64

In [33]:
df_sample["Errors?"] = df_sample["Errors?"].fillna("No Errors")

In [35]:
df_sample.head(20)

,User,Card,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,day_of_week,hour,is_weekend,time_of_day
18199893,1470,0,59.18,Chip Transaction,-6853385250336487907,Harwood,MD,20776.0,5813,No Errors,No,2,0,0,NaN
9731325,822,1,280.91,Online Transaction,4241336128694185533,ONLINE,NaN,NaN,4814,No Errors,No,0,22,0,Evening
536687,41,3,-144.00,Swipe Transaction,190253443608377572,Hemet,CA,92543.0,3359,No Errors,No,0,7,0,Morning
13223840,1084,0,6.76,Chip Transaction,-7837310524365334241,Littleton,CO,80122.0,5300,No Errors,No,6,14,1,Afternoon
17070521,1384,0,9.17,Swipe Transaction,-5023497618971072366,Gardner,KS,66030.0,5812,No Errors,No,6,11,1,Morning
792843,55,3,1.36,Swipe Transaction,-6571010470072147219,Rego Park,NY,11374.0,5499,No Errors,No,5,9,1,Morning
8966297,776,0,97.81,Swipe Transaction,-6974082828836151610,Milwaukee,WI,53224.0,4900,No Errors,No,6,22,1,Evening
895801,66,0,23.25,Swipe Transaction,3675785629314646441,Gonzales,TX,78629.0,7349,No Errors,No,1,13,0,Afternoon
22902727,1880,2,486.70,Swipe Transaction,-3398248499422470718,Atlantic City,NJ,8401.0,7995,No Errors,No,4,1,0,Night
5845532,490,5,92.67,Chip Transaction,4722913068560264812,Pompano Beach,FL,33063.0,5411,No Errors,No,5,13,1,Afternoon


In [37]:
df_sample.isnull().sum()

User                   0
Card                   0
Amount                 0
Use Chip               0
Merchant Name          0
Merchant City          0
Merchant State    271549
Zip               287254
MCC                    0
Errors?                0
Is Fraud?              0
day_of_week            0
hour                   0
is_weekend             0
time_of_day        23510
dtype: int64

In [39]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2438690 entries, 18199893 to 23041253
Data columns (total 15 columns):
 #   Column          Dtype   
---  ------          -----   
 0   User            int64   
 1   Card            int64   
 2   Amount          float64 
 3   Use Chip        object  
 4   Merchant Name   int64   
 5   Merchant City   object  
 6   Merchant State  object  
 7   Zip             float64 
 8   MCC             int64   
 9   Errors?         object  
 10  Is Fraud?       object  
 11  day_of_week     int32   
 12  hour            int32   
 13  is_weekend      int32   
 14  time_of_day     category
dtypes: category(1), float64(2), int32(3), int64(4), object(5)
memory usage: 253.5+ MB


In [41]:
from sklearn.preprocessing import LabelEncoder

In [43]:
# Encode categorical features
label_enc = LabelEncoder()
df_sample["Use Chip"] = label_enc.fit_transform(df_sample["Use Chip"])
df_sample["Errors?"] = label_enc.fit_transform(df_sample["Errors?"])
df_sample["Is Fraud?"] = df_sample["Is Fraud?"].map({"No": 0, "Yes": 1})

df_sample["Merchant City"] = df_sample["Merchant City"].fillna("Unknown")
df_sample["Merchant State"] = df_sample["Merchant State"].fillna("Unknown")
df_sample["Merchant City"] = label_enc.fit_transform(df_sample["Merchant City"])
df_sample["Merchant State"] = label_enc.fit_transform(df_sample["Merchant State"])

In [47]:
# Convert 'time_of_day' to numerical categories
time_of_day_mapping = {"Morning": 0, "Afternoon": 1, "Evening": 2, "Night": 3}
df_sample["time_of_day"] = df_sample["time_of_day"].astype(object)
df_sample["time_of_day"].fillna("Unknown", inplace=True)
df_sample["time_of_day"] = df_sample["time_of_day"].map(time_of_day_mapping)

C:\Users\prati\AppData\Local\Temp\ipykernel_11416\1796859451.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sample["time_of_day"].fillna("Unknown", inplace=True)


In [49]:
df_sample.head()

,User,Card,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,day_of_week,hour,is_weekend,time_of_day
18199893,1470,0,59.18,0,-6853385250336487907,4165,98,20776.0,5813,19,0,2,0,0,NaN
9731325,822,1,280.91,1,4241336128694185533,0,189,NaN,4814,19,0,0,22,0,2.0
536687,41,3,-144.00,2,190253443608377572,4256,24,92543.0,3359,19,0,0,7,0,0.0
13223840,1084,0,6.76,0,-7837310524365334241,5543,25,80122.0,5300,19,0,6,14,1,1.0
17070521,1384,0,9.17,2,-5023497618971072366,3585,85,66030.0,5812,19,0,6,11,1,0.0


In [52]:
from sklearn.preprocessing import StandardScaler

In [54]:
# Feature scaling for numerical columns
scaler = StandardScaler()
df_sample[["Amount", "hour", "MCC"]] = scaler.fit_transform(df_sample[["Amount", "hour", "MCC"]])

In [56]:
# New feature: High-value transaction flag (1 if Amount > 75th percentile)
df_sample["high_value_transaction"] = (df_sample["Amount"] > df_sample["Amount"].quantile(0.75)).astype(int)

# New feature: Late-night transaction flag (1 if transaction is between 10 PM - 5 AM)
df_sample["late_night_transaction"] = ((df_sample["hour"] >= 22) | (df_sample["hour"] <= 5)).astype(int)

In [58]:
# User behavior-based features
df_sample["user_transaction_count"] = df_sample.groupby("User")["User"].transform("count")
df_sample["card_transaction_count"] = df_sample.groupby("Card")["Card"].transform("count")
df_sample["user_merchant_count"] = df_sample.groupby(["User", "Merchant Name"])["Merchant Name"].transform("count")
df_sample["user_avg_amount"] = df_sample.groupby("User")["Amount"].transform("mean")
df_sample["card_avg_amount"] = df_sample.groupby("Card")["Amount"].transform("mean")
df_sample["above_avg_spending"] = (df_sample["Amount"] > df_sample["user_avg_amount"]).astype(int)
df_sample["unique_cards_per_user"] = df_sample.groupby("User")["Card"].transform("nunique")


In [60]:
df_sample.head()

,User,Card,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,...,time_of_day,high_value_transaction,late_night_transaction,user_transaction_count,card_transaction_count,user_merchant_count,user_avg_amount,card_avg_amount,above_avg_spending,unique_cards_per_user
18199893,1470,0,0.188576,0,-6853385250336487907,4165,98,20776.0,0.285854,19,...,NaN,0,1,938,868659,229,0.289476,0.005003,0,2
9731325,822,1,2.885724,1,4241336128694185533,0,189,NaN,-0.850532,19,...,2.0,1,1,1599,649398,19,0.525108,0.002522,1,5
536687,41,3,-2.282929,2,190253443608377572,4256,24,92543.0,-2.505628,19,...,0.0,0,1,1735,280492,3,-0.118051,-0.007847,0,5
13223840,1084,0,-0.449067,0,-7837310524365334241,5543,25,80122.0,-0.297695,19,...,1.0,0,1,1742,868659,58,0.264983,0.005003,0,1
17070521,1384,0,-0.419751,2,-5023497618971072366,3585,85,66030.0,0.284716,19,...,0.0,0,1,1050,868659,35,0.136923,0.005003,0,3


In [62]:
df_sample["Zip"].fillna(df_sample["Zip"].mode()[0], inplace=True)
df_sample["time_of_day"].fillna(df_sample["time_of_day"].mode()[0], inplace=True)

C:\Users\prati\AppData\Local\Temp\ipykernel_11416\2284214432.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sample["Zip"].fillna(df_sample["Zip"].mode()[0], inplace=True)
C:\Users\prati\AppData\Local\Temp\ipykernel_11416\2284214432.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

In [64]:
df_sample.isnull().sum()

User                      0
Card                      0
Amount                    0
Use Chip                  0
Merchant Name             0
Merchant City             0
Merchant State            0
Zip                       0
MCC                       0
Errors?                   0
Is Fraud?                 0
day_of_week               0
hour                      0
is_weekend                0
time_of_day               0
high_value_transaction    0
late_night_transaction    0
user_transaction_count    0
card_transaction_count    0
user_merchant_count       0
user_avg_amount           0
card_avg_amount           0
above_avg_spending        0
unique_cards_per_user     0
dtype: int64

In [66]:
df_sample.head()

,User,Card,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,...,time_of_day,high_value_transaction,late_night_transaction,user_transaction_count,card_transaction_count,user_merchant_count,user_avg_amount,card_avg_amount,above_avg_spending,unique_cards_per_user
18199893,1470,0,0.188576,0,-6853385250336487907,4165,98,20776.0,0.285854,19,...,0.0,0,1,938,868659,229,0.289476,0.005003,0,2
9731325,822,1,2.885724,1,4241336128694185533,0,189,98516.0,-0.850532,19,...,2.0,1,1,1599,649398,19,0.525108,0.002522,1,5
536687,41,3,-2.282929,2,190253443608377572,4256,24,92543.0,-2.505628,19,...,0.0,0,1,1735,280492,3,-0.118051,-0.007847,0,5
13223840,1084,0,-0.449067,0,-7837310524365334241,5543,25,80122.0,-0.297695,19,...,1.0,0,1,1742,868659,58,0.264983,0.005003,0,1
17070521,1384,0,-0.419751,2,-5023497618971072366,3585,85,66030.0,0.284716,19,...,0.0,0,1,1050,868659,35,0.136923,0.005003,0,3


In [68]:
# Save the processed dataset
df_sample.to_csv("processed_data.csv", index=False)

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [76]:
print(df_sample.columns)  # Check all columns

Index(['User', 'Card', 'Amount', 'Use Chip', 'Merchant Name', 'Merchant City',
       'Merchant State', 'Zip', 'MCC', 'Errors?', 'Is Fraud?', 'day_of_week',
       'hour', 'is_weekend', 'time_of_day', 'high_value_transaction',
       'late_night_transaction', 'user_transaction_count',
       'card_transaction_count', 'user_merchant_count', 'user_avg_amount',
       'card_avg_amount', 'above_avg_spending', 'unique_cards_per_user'],
      dtype='object')


In [78]:
Y= df_sample['Is Fraud?']

In [82]:
X= df_sample.drop(columns=['Is Fraud?'])

In [88]:
X_train, X_test, Y_train,Y_test= train_test_split(X,Y, test_size=.02, random_state=42)

In [90]:
model= LogisticRegression()

In [92]:
model.fit(X_train, Y_train)

LogisticRegression()

In [96]:
y_pred= model.predict(X_test)

In [100]:
acc= accuracy_score(Y_test, y_pred)

In [102]:
print(acc)

0.5372944601632017


In [112]:
# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define models and hyperparameters
models = {
    "Logistic Regression": (LogisticRegression(), {
        "C": [0.01, 0.1, 1, 10],
        "solver": ["liblinear"]
    }),
    "Random Forest": (RandomForestClassifier(), {
        "n_estimators": [50, 100, 200],
        "max_depth": [5, 10, 20],
        "min_samples_split": [2, 5]
    }),
    "XGBoost": (XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 0.2],
        "max_depth": [3, 6, 9]
    })
}

# Train and evaluate each model
best_models = {}
for name, (model, params) in models.items():
    print(f"\n🔍 Training {name} with Hyperparameter Tuning...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, params, cv=3, scoring="roc_auc", n_jobs=-1)
    grid_search.fit(X_train_scaled, Y_train)

    # Save best model
    best_model = grid_search.best_estimator_
    best_models[name] = best_model

    # Predict on test set
    y_pred = best_model.predict(X_test_scaled)
    y_prob = best_model.predict_proba(X_test_scaled)[:, 1]

    # Evaluate model
    print(f"\n📊 Best Model for {name}: {grid_search.best_params_}")
    print(f"✅ Accuracy: {accuracy_score(Y_test, y_pred):.4f}")
    print(f"✅ ROC-AUC Score: {roc_auc_score(Y_test, y_prob):.4f}")
    print(f"✅ Classification Report:\n{classification_report(Y_test, y_pred)}")

NameError: name 'RandomForestClassifier' is not defined